In [ ]:
import sys,os
sys.path.append("..")
import django
django.setup()
import pandas as pd
from io import StringIO
import requests
import datetime
from crawlers.finlab.data_process_tools import last_month,year_transfer
from crawlers.models import *
from crawlers.finlab.pioneers import *
from crawlers.finlab.import_tools import *

In [ ]:
class CrawlInsiderHoldDetailTW:
    def __init__(self,date):
        self.date=date
        self.url_date=last_month(self.date)
        self.target_name = "董事、監察人、經理人及大股東持股月明細"
        self.sub_market = ["sii", "otc", "rotc"]

    def check_trade_day(self):
        month_date=self.date.strftime('%Y-%m-10')
        df=MonthlyRevenueTW.objects.filter(date=month_date).order_by('stock_id').values('stock_id')
        df=[v['stock_id'] for v in df]        
        if len(df)>0:
            return df
        else:
            return None
        
    def detail(self,stock_id,year,month):
        url='https://mops.twse.com.tw/mops/web/ajax_stapap1'
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko)'
                          ' Chrome/39.0.2171.95 Safari/537.36'}
        res = requests.post(url, {
            'firstin':'true',
            'year':year,
            'month':month,
            'co_id':stock_id,
            'step':'0'
        }, headers=headers
                           )
        try:
            df=pd.read_html(res.text)
        except Exception as e :
            print(e)
            return None
        try:
            table_loc=[i for i in range(len(df)) if len(df[i])>10 ][0]
        except Exception as e :
            print(e)
            return None  
        df=df[table_loc]
        df=df[df[0]!='職稱']
        df=df.astype(str)
        df=df.apply(lambda s: s.str.replace("%", ""))
        df.columns=['title','name','act_hold','hold','pledge',
                           'pledge_ratio','family_hold','family_pledge','family_pledge_ratio']
        df.iloc[:, 2:] = df.iloc[:, 2:].apply(lambda s: pd.to_numeric(s, errors="coerce"))
        df['stock_id']=stock_id
        df['date'] = datetime.date(self.date.year, self.date.month, 20)
#         AddToSQL.add_to_sql(InsiderHoldDetailTW, df, pk_columns=['stock_id', 'date', 'title', 'name'],
#                             jump_update=True)
        return df

    def crawl_main(self):
        crawl_list = self.check_trade_day()
        year=self.url_date.year-1911
        month=self.url_date.month
        # check holiday
        if crawl_list is not None:
            # check is new or old process
#             new_obj = InsiderHoldDetailTW.objects.filter(date=self.start_date_str)
#             if len(new_obj) > 0:
#                 table_last_day = table_latest_date(engine, InsiderHoldDetailTW._meta.db_table)
#                 finish_obj = InsiderHoldDetailTW.objects.filter(date=table_last_day)
#                 last_stock_id = finish_obj[len(finish_obj) - 1].stock_id
#                 crawl_list = crawl_list[crawl_list.index(last_stock_id) + 1:]
            data=[]
            for stock_id in crawl_list:
                print(stock_id)
                data.append(self.detail(stock_id,year,month))
                time.sleep(3)
            return data
        else:
            pass

In [ ]:
C=CrawlInsiderHoldDetailTW(datetime.datetime(2020,5,1))
C.crawl_main()

In [1]:
import pandas as pd
from io import StringIO
import requests
import datetime

def detail(stock_id,year,month):

    url='https://mops.twse.com.tw/mops/web/ajax_stapap1'
    res = requests.post(url, {
        'firstin':'true',
        'year':year,
        'month':month,
        'co_id':stock_id,
        'step':'0'
    }
                       )
    try:
        df=pd.read_html(res.text)
    except ValueError :
        return None
    try:
        table_loc=[i for i in range(len(df)) if len(df[i])>10 ][0]
    except IndexError :
        return None        
    print(table_loc)
    df=df[table_loc]
    df=df[df[0]!='職稱']
    df=df.astype(str)
    df=df.apply(lambda s: s.str.replace("%", ""))
    df.columns=['title','name','act_hold','hold','pledge',
                       'pledge_ratio','family_hold','family_pledge','family_pledge_ratio']
    df.iloc[:, 2:] = df.iloc[:, 2:].apply(lambda s: pd.to_numeric(s, errors="coerce"))
    df['stock_id']=stock_id
    return df

detail('1101',109,4)

3


,title,name,act_hold,hold,pledge,pledge_ratio,family_hold,family_pledge,family_pledge_ratio,stock_id
2,董事長本人,嘉利實業股份有限公司,2.45383e+06,2.88845e+06,0,0,0,0,0,1101
3,董事長之法人代表人,張安平,0,184304,0,0,3.60177e+06,0,0,1101
4,董事本人,信和投資股份有限公司,1.07016e+07,1.6136e+07,0,0,0,0,0,1101
5,董事之法人代表人,張啟文,0,3.21604e+06,0,0,0,0,0,1101
6,董事本人,財團法人辜公亮文教基金會,2.30891e+06,3.01786e+06,0,0,0,0,0,1101
...,...,...,...,...,...,...,...,...,...,...
71,協理本人,陳光熙,0,74907,0,0,0,0,0,1101
74,副理本人,吳志,0,75309,0,0,3531,0,0,1101
75,副理本人,林照貴,0,53505,0,0,0,0,0,1101
76,副理本人,蕭立奇,0,3000,0,0,0,0,0,1101


In [ ]:
url='https://mops.twse.com.tw/mops/web/ajax_stapap1'
res = requests.post(url, {
    'firstin':'true',
    'year':'109',
    'month':'4',
    'co_id':'1101',
#     'TYPEK': 'sii',
    'step':'0'

})
res.text
# df=pd.read_html(res.text)
# table_loc=[i for i in range(len(df)) if len(df[i])>10 ][0]
# df=df[table_loc]
# df

In [ ]:
df=MonthlyRevenueTW.objects.filter(date='2020-4-10').order_by('stock_id').values('stock_id')
df=[v['stock_id'] for v in df]
df

In [ ]:
import datetime
datetime.datetime(2020,5,1).strftime('%Y-%m-10')